In [37]:
# Import necessary libraries
import requests
import pandas as pd
import hashlib

# Define FastAPI endpoints
base_url = 'http://localhost:8000' 


In [19]:
# Load the test data
test_data_path = '../data/test.csv'
test_data = pd.read_csv(test_data_path)

In [64]:
def hash_tweet(text):
    # Create a SHA-256 hash of the tweet text and return the first 32 characters
    hash_object = hashlib.sha256(text.encode())
    return hash_object.hexdigest()[:8]


In [65]:
# Prepare data for single prediction
def prepare_single_prediction_data(row):
    return {'text': row['tweets']}

# Prepare data for batch prediction
def prepare_batch_prediction_data(df):
    # Hash each tweet to create its ID
    tweet_ids = [hash_tweet(tweet) for tweet in df['tweets']]
    
    # Convert DataFrame columns to lists
    texts = df['tweets'].tolist()

    # Prepare data for the request
    return {
        "tweet_ids": tweet_ids,
        "texts": texts
    }



In [66]:
# Make a single prediction request
def predict_single(text):
    tweet_id = hash_tweet(text)
    
    # Prepare the request data
    data = {
        'tweet_id': tweet_id,
        'text': text
    }
    # Send the POST request to the /predict endpoint
    response = requests.post(f'{base_url}/predict', json=data)
    # Return the response as JSON
    return response.json()


In [75]:
# Make a batch prediction request
def predict_batch(df):
    # Prepare the batch prediction data
    batch_data = prepare_batch_prediction_data(df)
    # Make the POST request
    response = requests.post(f'{base_url}/predict', json=batch_data)
    
    # Return the response JSON
    return response.json()

In [76]:
# Example: Predict a single tweet
single_prediction = predict_single(test_data['tweets'].iloc[6])
print("Single Prediction:", single_prediction)

Single Prediction: {'tweet_id': '2582afe1', 'tweet': 'well that escalated quickly mtb motor woowooelectricsta', 'user': 'some_user', 'likes': 0, 'retweets': 0, 'linreg_prob': 0.37973127429662035, 'linreg_result': 0, 'cnn_prob': None, 'cnn_result': None}


In [69]:
for i in range(5):
    single_prediction = predict_single(test_data['tweets'].iloc[i])
    print(f"Tweet: {test_data['tweets'][i][:15]}\nSingle Prediction for tweet {i}:\n", single_prediction)

Tweet: i am trying to 
Single Prediction for tweet 0:
 {'logreg_prediction': 0, 'logreg_prob': 0.060409772267669935}
Tweet: congratulations
Single Prediction for tweet 1:
 {'logreg_prediction': 1, 'logreg_prob': 0.6791508577772777}
Tweet: say i fail agai
Single Prediction for tweet 2:
 {'logreg_prediction': 1, 'logreg_prob': 0.7988666535347574}
Tweet: amennn if your 
Single Prediction for tweet 3:
 {'logreg_prediction': 0, 'logreg_prob': 0.08690736075482393}
Tweet: if maybe just a
Single Prediction for tweet 4:
 {'logreg_prediction': 1, 'logreg_prob': 0.9284949056037131}


In [78]:
# Example: Predict in batch
batch_prediction = predict_batch(test_data[:4])
print("Batch Prediction Results:")
batch_prediction

Batch Prediction Results:


{'results': [{'tweet_id': 'ea3ec30d',
   'tweet': 'i am trying to get one i have emailed channel asking them for a source that would be acceptable by wikipedia s standards hopefully they will reply in the next working day or two',
   'user': 'some_user',
   'likes': 0,
   'retweets': 0,
   'linreg_prob': 0.060409772267669935,
   'linreg_result': 0,
   'cnn_prob': 0.9787822067737579,
   'cnn_result': 0},
  {'tweet_id': '6251ca46',
   'tweet': 'congratulations congratulations floquenbeam you have won the asshole of the year award you have won this award by saying mean things about and to other people in order to claim your award you must call asshole once again congratulations on your win',
   'user': 'some_user',
   'likes': 0,
   'retweets': 0,
   'linreg_prob': 0.6791508577772777,
   'linreg_result': 1,
   'cnn_prob': 0.9554113745689392,
   'cnn_result': 1},
  {'tweet_id': 'e82c2629',
   'tweet': 'say i fail again and you are being reported you think that you are the hot metal shot th